In [7]:
from src.data import CostomerDataset
from src.model.mlp import Model
from src.utils import convert_category_into_integer

import pandas as pd
import numpy as np
import random
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
from torch.utils.data import DataLoader


def main(configs):
    data = pd.read_csv('./data/train.csv')
    data = data.drop(columns=[
        'CustomerID',  # 구분용도임
        'NotNewCellphoneUser', # NewCell...이랑 중복
        'BlockedCalls', 
        'PrizmCode', # 인구통계 세분화 코드라 삭제
        'TruckOwner', # 자동차 오토바이 유무라서 삭제
        'OwnsMotorcycle',
        'OwnsComputer', # 컴퓨터 유무 삭제
        'OffPeakCallsInOut',
        'OptOutMailings',
        'NonUSTravel',# 미국 여행여부 삭제
        'AdjustmentsToCreditRating', # 신용등급
        'ActiveSubs',
    ])
    data = data.dropna()

    # 나이 0 삭제
    data = data[data['AgeHH1']>18]
    data = data[data['AgeHH2']>18]

    # 인트타입으로 바꿔서 구분
    data['Churn'] = np.where(data['Churn']=="Yes", 1, 0)
    data['DroppedCalls'] = data['DroppedCalls'].astype(int)

    # 나이 구분
    data.AgeHH1 = np.where(
        data.AgeHH1 < 20,18,
        np.where(data.AgeHH1 < 30, 20, 
        np.where(data.AgeHH1 < 40, 30,
        np.where(data.AgeHH1 < 50, 40,
        np.where(data.AgeHH1 < 60, 50, 60)))))
    data.AgeHH2 = np.where(
        data.AgeHH2 < 20,18,
        np.where(data.AgeHH2 < 30, 20, 
        np.where(data.AgeHH2 < 40, 30,
        np.where(data.AgeHH2 < 50, 40,
        np.where(data.AgeHH2 < 60, 50, 60)))))

    data.CreditRating = data.CreditRating.str[0].astype(int) # 신용등급 숫자로 변경


    # 1. 고객 충성도 레이블 (Customer Loyalty)
    # 'MonthsInService'와 'RetentionCalls'을 기반으로 고객이 얼마나 오랜 기간 동안 서비스에 머물렀는지, 그리고 고객 유지 노력의 결과를 반영
    data['CustomerLoyalty'] = np.where(
        (data['MonthsInService'] > data['MonthsInService'].mean()) &
        (data['RetentionCalls'] > 0), 
        'High Loyalty', 'Low Loyalty'
    )

    # 2. 서비스 사용 기간 레이블 (Equipment Usage Duration)
    # 'CurrentEquipmentDays'를 사용해 서비스를 얼마나 사용하고 있는지 반영
    data['EquipmentUsageDuration'] = np.where(
        data['CurrentEquipmentDays'] > data['CurrentEquipmentDays'].mean(), 
        'Long-Term Equipment User', 'Short-Term Equipment User'
    )

    # 3. 요금 부담 레이블 (Charge Burden)
    # 'TotalRecurringCharge'를 기준으로 요금 부담이 높은지 낮은지를 분류
    data['ChargeBurden'] = np.where(
        data['TotalRecurringCharge'] > data['TotalRecurringCharge'].mean(), 
        'High Charge', 'Low Charge'
    )

    # 4. 신용 등급 레이블 (Credit Rating Category)
    # 'CreditRating'을 기준으로 신용 등급을 두 그룹으로 나눔
    data['CreditCategory'] = np.where(
        data['CreditRating'] > data['CreditRating'].median(), 
        'High Credit', 'Low Credit'
    )

    # 5. 구매 및 메일 응답 행동 (Purchase and Mail Response Behavior)
    # 'BuysViaMailOrder'와 'RespondsToMailOffers'를 결합하여 고객의 마케팅 참여도 파악
    data['MarketingEngagement'] = np.where(
        (data['BuysViaMailOrder'] == 'Yes') & (data['RespondsToMailOffers'] == 'Yes'), 
        'Fully Engaged',
        np.where(
            (data['BuysViaMailOrder'] == 'Yes') | (data['RespondsToMailOffers'] == 'Yes'), 
            'Partially Engaged', 'Not Engaged'
        )
    )

    # 6. 핸드셋 웹 사용 가능 여부 (Handset Web Capability)
    # 'HandsetWebCapable'을 사용하여 핸드셋이 웹 사용 가능한지 여부를 분류
    data['HandsetWebCapability'] = np.where(
        data['HandsetWebCapable'] == 'Yes', 
        'WebCapable', 'NonWebCapable'
    )
    category_columns = ['ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable',
                        'RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers',
                        'HasCreditCard','NewCellphoneUser','HandsetPrice','MadeCallToRetentionTeam',
                        'Occupation','MaritalStatus','CustomerLoyalty','EquipmentUsageDuration','ChargeBurden',
                        'CreditCategory','MarketingEngagement','HandsetWebCapability']
    
    data, _ = convert_category_into_integer(data, (category_columns))
    data = data.astype(np.float32)

    # Train/Validation/Test Split
    _, temp = train_test_split(data, test_size=0.4, random_state=seed)
    _, test = train_test_split(temp, test_size=0.5, random_state=seed)

    # Dataset과 DataLoader 설정
    test_dataset = CostomerDataset(test)

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=configs.get('batch_size'),
    )

    configs.update({'input_dim': len(data.columns)-1})
    
    # 모델 로드
    model = Model(configs)
    model_state_dict = torch.load("./model/mlp.pth")
    model.load_state_dict(model_state_dict)
    model.eval()  # 평가 모드로 전환

    # prediction
    preds = []
    gts = []
    for batch in test_dataloader:
        X = batch.get('X')
        y = batch.get('y')
        with torch.no_grad():  # 예측 시에는 그래디언트 필요 없음
            pred = model(X)
            preds.append(pred.sigmoid().round())
            gts.append(y)

    # 텐서 리스트를 합침
    preds = torch.cat(preds)
    gts = torch.cat(gts)

    print(classification_report(gts, preds))

if __name__ == '__main__':
    # 사용 가능한 GPU가 있는 경우 'cuda', 그렇지 않으면 'cpu' 사용
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # hyperparameter
    with open('./configs.json', 'r') as file:
        configs = json.load(file)
    configs.update({'device': device})

    # seed 설정
    seed = configs.get('seed')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # CUDA 설정
    if device == 'cuda':
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
    
    main(configs)


C:\Users\USER\AppData\Local\Temp\ipykernel_25044\2570488946.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load("./model/mlp.pth")


              precision    recall  f1-score   support

         0.0       0.72      1.00      0.84      3428
         1.0       0.00      0.00      0.00      1310

    accuracy                           0.72      4738
   macro avg       0.36      0.50      0.42      4738
weighted avg       0.52      0.72      0.61      4738

